<a href="https://colab.research.google.com/github/faisalnawazmir/Econometrics-ML_for_Finance/blob/main/ayesha__Fama_q_Factor_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fama French Q Factor Model  using split data
2000-2005,2006-2008,2009-2012,2013-2016,2017-2019 (5 splits)

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm 
from tabulate import tabulate
import time
np.random.seed(12345)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [148]:
sb='2017-2019'# split time

In [149]:
#copying data from excel file 25 portfoli
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
file = r'/gdrive/My Drive/data/q_q5_data_sheet.xlsx'
df = pd.read_excel(file, sheet_name=sb)
df.head()

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


,Date,MKt,Me,ROE,I/A,Reg,Unnamed: 6,Unnamed: 7,Unnamed: 8,Date.1,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24,p25
0,2017-01-01,0.014843,-0.011870,-0.039684,-0.064430,-0.043967,NaN,NaN,NaN,2017-01-01,0.028101,0.035439,0.029060,0.037174,0.017918,0.018859,0.004814,0.009151,0.015900,0.017504,0.004397,0.005888,-0.002140,-0.008795,-0.011016,0.008446,0.011779,0.010971,0.009399,0.004280,-0.003078,-0.001203,-0.009265,-0.003336,-0.005026
1,2017-01-02,-0.009441,0.027685,0.053202,0.024197,0.033439,NaN,NaN,NaN,2017-01-02,0.019074,0.036646,0.012164,0.015344,0.013320,-0.004486,-0.032764,-0.025830,-0.011593,-0.012138,0.001202,-0.012525,-0.005592,-0.002651,-0.000181,-0.000252,0.007122,0.007416,0.009492,0.010329,0.007952,0.002430,-0.016385,-0.003015,-0.011921
2,2017-01-03,-0.012673,0.006536,0.040037,0.031361,0.052201,NaN,NaN,NaN,2017-01-03,-0.003348,-0.001279,0.002225,0.004086,0.018085,0.012355,0.012143,0.012874,0.018864,0.033527,0.032003,0.030343,0.037981,0.042181,0.038091,0.039651,0.040206,0.032929,0.033645,0.022575,-0.004886,0.021728,0.015195,0.011838,0.000796
3,2017-01-04,0.018650,-0.018119,0.011972,-0.023361,-0.019770,NaN,NaN,NaN,2017-01-04,0.036349,0.033226,0.030330,0.031842,0.003802,0.001619,0.017445,0.012029,0.015243,0.022545,0.023613,0.014514,0.015315,0.024276,0.026057,0.018759,-0.001152,-0.003580,0.000829,-0.008255,-0.013187,-0.025396,-0.041960,-0.047804,-0.060721
4,2017-01-05,0.020995,0.030408,0.037446,0.026583,0.074397,NaN,NaN,NaN,2017-01-05,0.147400,0.101580,0.063436,0.048036,0.064690,0.066716,0.045507,0.031786,0.018769,0.008268,0.010219,-0.007003,-0.001505,-0.005142,-0.001723,-0.006371,-0.006311,-0.016328,-0.020156,-0.014850,-0.003571,0.009088,0.006389,-0.001668,-0.004995


In [150]:
# dropping passed columns
df.drop(["Unnamed: 6","Unnamed: 7", "Unnamed: 8","Date.1"], axis = 1, inplace = True)
df.head()
#You must specify the axis argument. default is axis = 0 which is rows columns is axis = 1.

,Date,MKt,Me,ROE,I/A,Reg,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24,p25
0,2017-01-01,0.014843,-0.011870,-0.039684,-0.064430,-0.043967,0.028101,0.035439,0.029060,0.037174,0.017918,0.018859,0.004814,0.009151,0.015900,0.017504,0.004397,0.005888,-0.002140,-0.008795,-0.011016,0.008446,0.011779,0.010971,0.009399,0.004280,-0.003078,-0.001203,-0.009265,-0.003336,-0.005026
1,2017-01-02,-0.009441,0.027685,0.053202,0.024197,0.033439,0.019074,0.036646,0.012164,0.015344,0.013320,-0.004486,-0.032764,-0.025830,-0.011593,-0.012138,0.001202,-0.012525,-0.005592,-0.002651,-0.000181,-0.000252,0.007122,0.007416,0.009492,0.010329,0.007952,0.002430,-0.016385,-0.003015,-0.011921
2,2017-01-03,-0.012673,0.006536,0.040037,0.031361,0.052201,-0.003348,-0.001279,0.002225,0.004086,0.018085,0.012355,0.012143,0.012874,0.018864,0.033527,0.032003,0.030343,0.037981,0.042181,0.038091,0.039651,0.040206,0.032929,0.033645,0.022575,-0.004886,0.021728,0.015195,0.011838,0.000796
3,2017-01-04,0.018650,-0.018119,0.011972,-0.023361,-0.019770,0.036349,0.033226,0.030330,0.031842,0.003802,0.001619,0.017445,0.012029,0.015243,0.022545,0.023613,0.014514,0.015315,0.024276,0.026057,0.018759,-0.001152,-0.003580,0.000829,-0.008255,-0.013187,-0.025396,-0.041960,-0.047804,-0.060721
4,2017-01-05,0.020995,0.030408,0.037446,0.026583,0.074397,0.147400,0.101580,0.063436,0.048036,0.064690,0.066716,0.045507,0.031786,0.018769,0.008268,0.010219,-0.007003,-0.001505,-0.005142,-0.001723,-0.006371,-0.006311,-0.016328,-0.020156,-0.014850,-0.003571,0.009088,0.006389,-0.001668,-0.004995


In [151]:
#N = 25  # of firms in an economy, here # of portfolios, here len(data)
T = len(df)  # length of periods # here 1136#df.columns.array
#  data
data = df.loc[:,'p1':'p25']
#data.shape#(T,N)
#x,y=data.shape
#N=y#  # of firms in an economy, here # of portfolios, here len(data)
#T = data.shape[0]
N = data.shape[1]

In [152]:
# simulate factor loadings (ie, coefficients)
b_mkt = np.zeros((1,N)) # true loadings for market factor
b_me = np.zeros((1,N)) # true loadings for size factor
b_roe = np.zeros((1,N)) # true loadings for value factor
b_ia = np.zeros((1,N)) # true loadings for value factor
#b_reg = np.zeros((1,N)) # true loadings for value factor

In [153]:
#  factor returns
mkt = df['MKt'] # market factor
me = df['Me'] # size factor
roe = df['ROE'] # value factor
ia = df['I/A'] # market factor
#reg=df['Reg']# umd is growth factor


In [154]:
#data.iloc[:,1]
fctrs = df[["MKt","Me","ROE","I/A"]]
fctrs.head()


,MKt,Me,ROE,I/A
0,0.014843,-0.011870,-0.039684,-0.064430
1,-0.009441,0.027685,0.053202,0.024197
2,-0.012673,0.006536,0.040037,0.031361
3,0.018650,-0.018119,0.011972,-0.023361
4,0.020995,0.030408,0.037446,0.026583


In [155]:
no_factors=fctrs.shape[1]

In [156]:
# Step 1: time series regression firm by firm
#fctrs = np.c_[mkt, smb, hml]
# of factors
b = np.zeros(no_factors)
for i in range(N): # firm-by-firm time series regression
  model = LinearRegression()
  model.fit(fctrs,data.iloc[:,i])
  b = np.c_[b,model.coef_]

b  = np.transpose(b[:,1:])
b.shape
#b should be 0.1, 0.2

(25, 4)

In [157]:
#b is bhat
#b
#b.shape# (N,#factors)
df_bhat = pd.DataFrame(b)
df_bhat.columns = ['b_01','b_02','b_03','b_04']# change according factos
#print(df_bhat.head())

from google.colab import  drive
df_bhat.to_csv('/gdrive/My Drive/data/bhat_ayesha_q_factors_split_'+sb+'.csv')

In [158]:
# Step 2: cross sectional regression time-by-time
f = np.zeros(no_factors)
for i in range(T): # time-by-time cross sectional regression
  model = LinearRegression()
  model.fit(b,data.iloc[i,:])
  f = np.c_[f,model.coef_]

f = np.transpose(f[:,1:])
df_gamma = pd.DataFrame(f)
df_gamma.columns = ['mkt','me','roe','ia']
print(df_gamma.describe())
print('Median estimate:', np.median(f,axis=0))
print('Standard errors (without Shanken correction):', np.std(f,axis=0)/T**.5)

             mkt         me        roe         ia
count  36.000000  36.000000  36.000000  36.000000
mean   -0.003205   0.001696   0.005347  -0.008050
std     0.062606   0.025068   0.064274   0.059800
min    -0.147933  -0.048890  -0.177596  -0.176684
25%    -0.034397  -0.012241  -0.040714  -0.023410
50%    -0.009298  -0.000825   0.008559  -0.002989
75%     0.028669   0.014738   0.053682   0.026317
max     0.197505   0.073945   0.159690   0.111090
Median estimate: [-0.00929831 -0.00082529  0.00855892 -0.00298916]
Standard errors (without Shanken correction): [0.01028839 0.00411954 0.01056252 0.00982734]


## Writing output to the text file

In [159]:
file_output=r'/gdrive/My Drive/data/output_factor_split_'+sb+'.txt'
desc=repr(df_gamma.describe())
median_est=repr(np.median(f,axis=0))
std_err=repr(np.std(f,axis=0)/T**.5)

Using the with statement is better practice, it automatically closes the file even if the code encounters an exception. The code will run everything in the indent block then close the file object. 

In [160]:
mu=np.median(f,axis=0)
sigma=(np.std(f,axis=0))/T**.5
tstat=mu/sigma
tstat
t_stats=repr(tstat)
#tstat<1.96

In [161]:
# Write lines to file

with open(file_output, 'w+') as writefile:
    writefile.write("Results of Table  A\n")
    writefile.write(desc+"\n")
    writefile.write("Median estimate:"+"\t"+median_est+"\n")
    writefile.write("Standard errors (without Shanken correction):\t"+std_err+"\n")
    writefile.write("T stats: \t"+t_stats+"\n")
    writefile.write("***********\n\n")


In [162]:
# Check whether write to file

with open(file_output, 'r') as testwritefile:
    print(testwritefile.read())

Results of Table  A
             mkt         me        roe         ia
count  36.000000  36.000000  36.000000  36.000000
mean   -0.003205   0.001696   0.005347  -0.008050
std     0.062606   0.025068   0.064274   0.059800
min    -0.147933  -0.048890  -0.177596  -0.176684
25%    -0.034397  -0.012241  -0.040714  -0.023410
50%    -0.009298  -0.000825   0.008559  -0.002989
75%     0.028669   0.014738   0.053682   0.026317
max     0.197505   0.073945   0.159690   0.111090
Median estimate:	array([-0.00929831, -0.00082529,  0.00855892, -0.00298916])
Standard errors (without Shanken correction):	array([0.01028839, 0.00411954, 0.01056252, 0.00982734])
T stats: 	array([-0.90376799, -0.20033559,  0.8103104 , -0.30416756])
***********




In [163]:
f.shape # (T,#factors)# f or df I assume is gamma hat
df_gamma.to_csv('/gdrive/My Drive/data/ayesha_gamma_q_factors_split_'+sb+'.csv')

In [164]:
#df.head()#f and df are same, gamma hat

In [165]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import statsmodels.stats.api as sms

In [166]:
data_a=data.to_numpy()
fctrs_a=fctrs.to_numpy()

In [167]:
# Compute standard error with bootstrapping (Shanken correction unnecessary)
# One simple bootstrapping 

indices = np.random.randint(low=0,high=T, size=T)
new_data = data_a[indices,:]
new_fctrs = fctrs_a[indices,:]
xx = indices[0]
print(xx)
print(new_data[0,:])
print(data_a[xx,:])


20
[-0.05946895 -0.05426755 -0.04256458 -0.04080319 -0.04496213 -0.0302427
 -0.02068397 -0.01526257 -0.04901512 -0.03736265 -0.04454685 -0.04624192
 -0.0474963  -0.05381432 -0.05456073 -0.05593763 -0.06128708 -0.06611241
 -0.06745523 -0.0698133  -0.04202707 -0.04092071 -0.03374047 -0.0329572
 -0.03130393]
[-0.05946895 -0.05426755 -0.04256458 -0.04080319 -0.04496213 -0.0302427
 -0.02068397 -0.01526257 -0.04901512 -0.03736265 -0.04454685 -0.04624192
 -0.0474963  -0.05381432 -0.05456073 -0.05593763 -0.06128708 -0.06611241
 -0.06745523 -0.0698133  -0.04202707 -0.04092071 -0.03374047 -0.0329572
 -0.03130393]


In [168]:
def efhats(data,fctrs): # function to compute average estimated factor return
  T = data.shape[0]
  N = data.shape[1]
  # Step 1: Time series regression
  x = np.c_[np.ones(T),fctrs]
  M = np.linalg.inv(x.T @ x) @ x.T
  bhat = (M @ data)[1:,:].T

  # Step 2: cross sectional regression
  # Include other control variables here (eg, firm characteristics)
  x = np.c_[np.ones(N),bhat]
  M = np.linalg.inv(x.T @ x) @ x.T
  fhat = (M @ data.T)[1:,:].T
  df = pd.DataFrame(fhat)
  df.columns = ['mkt','me','roe','ia']
  E_fhat = np.average(fhat,axis=0)
  return E_fhat 


In [169]:
E_fhat = efhats(data_a,fctrs_a)
simn = 10**3 # of bootstrapping
t = time.time() # bootstrapping start time 
for i in range(simn): 
  indices = np.random.randint(low=0,high=T,size=T) # resample by rows
  new_data = data_a[indices,:] # resampled stock return data
  new_fctrs = fctrs_a[indices,:] # resampled factor return data
  E_fhat = np.c_[E_fhat,efhats(new_data,new_fctrs)]
  # to check bootstrapping status
  if i % 100 == 0: print(i)

print('Simulation duration:', time.time()-t, 'seconds.')
df = pd.DataFrame(E_fhat.T)
df.columns = ['mkt','me','roe','ia']
print(df.describe())

0
100
200
300
400
500
600
700
800
900
Simulation duration: 0.9723191261291504 seconds.
               mkt           me          roe           ia
count  1001.000000  1001.000000  1001.000000  1001.000000
mean     -0.003948     0.000868     0.001648    -0.003669
std       0.010909     0.004648     0.011661     0.012404
min      -0.037182    -0.018423    -0.039068    -0.045950
25%      -0.011729    -0.001905    -0.005907    -0.011864
50%      -0.003679     0.000867     0.001233    -0.003913
75%       0.003764     0.003545     0.008905     0.003879
max       0.032332     0.014363     0.052918     0.041956


In [170]:
print('FM regression esimate of expeced factor returns: \n', np.average(E_fhat.T,axis=0))
print('FM regression esimate of standard errors: \n', np.std(E_fhat.T,axis=0))

FM regression esimate of expeced factor returns: 
 [-0.00394761  0.00086818  0.00164802 -0.00366909]
FM regression esimate of standard errors: 
 [0.01090324 0.00464608 0.01165468 0.01239807]


In [171]:
#bootstrap t stats
mu_boot=np.average(E_fhat.T,axis=0)
sigma_boot=np.std(E_fhat.T,axis=0)
tstat_boot=mu_boot/sigma_boot
tstat_boot

array([-0.36205846,  0.18686334,  0.14140405, -0.29594035])

https://quant.stackexchange.com/questions/35781/how-to-interpret-the-grs-f-test-values
#grs test
https://stackoverflow.com/questions/24074481/fama-macbeth-regression-in-python-pandas-or-statsmodels

In [ ]:
pip install rpy2

In [40]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [172]:
%%R -i df -w 5 -h 5 --units in -r 200
# import df from global environment
# make default figure size 5 by 5 inches with 200 dpi resolution

install.packages("GRS.test", repos='http://cran.us.r-project.org', quiet=TRUE)
library(GRS.test)


In [173]:
%%R -i data,fctrs
GRS.MLtest(data, fctrs)

$GRS.stat
          GRS
[1,] 2.288777

$GRS.pval
           GRS
[1,] 0.1304919

$thetas
         [,1]
[1,] 2.931237

$theta
          [,1]
[1,] 0.4395644

$ratio
          [,1]
[1,] 0.1499587



In [174]:
%%R -o model -o coef  -o pval -o thetas -o theta -o ratio
model<-GRS.MLtest(data, fctrs)
coef<-model$GRS.stat
pval<-model$GRS.pval
thetas<-model$thetas
theta<-model$theta
ratio<-model$ratio

In [175]:
print(model)

$GRS.stat
          GRS
[1,] 2.288777

$GRS.pval
           GRS
[1,] 0.1304919

$thetas
         [,1]
[1,] 2.931237

$theta
          [,1]
[1,] 0.4395644

$ratio
          [,1]
[1,] 0.1499587




W statistic given in (7) of GRS (1989) <DOI:10.2307/1913625>

GRS.stat=GRS test statistic

GRS.pval=its p-value

theta=maximum Sharpe ratio of the K factor portfolios

thetas=slope of the efficient frontier based on all assets

ratio=theta/thetas, proportion of the potential efficiency

References
Gibbons, Ross, Shanken, 1989. A test of the efficiency of a given portfolio, Econometrica, 57,1121-
1152. <DOI:10.2307/1913625>


https://towardsdatascience.com/guide-to-r-and-python-in-a-single-jupyter-notebook-ff12532eb3ba

https://www.linkedin.com/pulse/interfacing-r-from-python-3-jupyter-notebook-jared-stufft/
how to switch R and python


In [176]:
grs_stat=repr(coef)
grs_pval=repr(pval)
grs_thets=repr(thetas)
grs_theta=repr(theta)
grs_ratio=repr(ratio)

In [177]:
#bootstrap t stat results
simulated_desc=repr(df.describe())
median_est_boot=repr(mu_boot)
std_err_boot=repr(sigma_boot)
t_stat_boot=repr(tstat_boot)

In [178]:
# Write lines to file

with open(file_output, 'a') as writefile:
    writefile.write("\n Results of Boot Strapping Table  B\n")
    writefile.write(simulated_desc+"\n")
    writefile.write("FM regression esimate of expeced factor returns:"+"\t"+median_est_boot+"\n")
    writefile.write("FM regression esimate of standard errors:\t"+std_err_boot+"\n")
    writefile.write("Tstats boot strapping: \t"+t_stat_boot+"\n")
    writefile.write("**********\n")
    writefile.write("\t GRS Test:\n")
    writefile.write("**********\n")
    writefile.write("GRS stat:\t"+grs_stat+"\n")
    writefile.write("GRS pvalue:\t"+grs_pval+"\n")
    writefile.write("GRS thetas:\t"+grs_thets+"\n")
    writefile.write("GRS theta:\t"+grs_theta+"\n")
    writefile.write("GRS ratio:\t"+grs_ratio+"\n")

In [50]:
#Erasing the file contents of a text file will result in an empty text file.
# file = open(file_output,"r+")
# file.truncate(0)
# file.close()